# Setup config for fake planet experiments

In [20]:
import json
from pathlib import Path
import numpy as np

from applefy.utils.file_handling import load_adi_data

from fours.utils.fwhm import get_fwhm
from fours.utils.data_handling import read_fours_root_dir

## 1. All dataset parameters

In [2]:
# Dataset setups (Name of the dataset, dit_psf, dit_science, ND scaling)
datasets = [ 
    ("HD2262_305_199_C-0065_C_", 0.02, 0.35, 1.0), #1
    ("HD7570_331_1101_C-0092_C_", 0.02, 0.35, 1.0), #2
    ("HD11171_332_1101_C-0092_C_", 0.02, 0.35, 1.0), #3
    ("HD22049_351_096_C-0679_A_", 0.0042560, 0.08, 1.0), #4
    ("HD22049_303_199_C-0065_C_", 0.0042560, 0.08, 1.0), #5
    ("HD38678_331_084_C-0396_A_", 0.2, 0.2, 55.87), #6
    ("HD40136_333_1101_C-0092_C_", 0.01, 0.3, 1.0), #7
    ("HD115892_143_1101_C-0092_E_", 0.008, 0.35, 1.0), #8
    ("HD169022_140_1101_C-0092_E_", 0.005, 0.35, 1.0), #9
    ("HD177724_231_091_C-0654_A_", 0.08, 0.2, 55.87), #10
    ("HD209952_196_089_C-0149_A_", 0.04, 0.2, 55.87), #11
]

In [10]:
# select one dataset
dataset_name, dit_psf, dit_science, nd_scaling  = datasets[1]
pixel_scale=0.02718
dataset_name

'HD7570_331_1101_C-0092_C_'

## 2. Load the dataset

In [11]:
root_dir = Path(read_fours_root_dir())

Data in the FOURS_ROOT_DIR found. Location: /fast/mbonse/s4


In [12]:
dataset_file = root_dir / Path("30_data/" + dataset_name + ".hdf5")
experiment_root_dir = root_dir / Path("70_results/x1_fake_planet_experiments/" + dataset_name)
experiment_root_dir.mkdir(exist_ok=True)

In [13]:
science_data, angles, raw_psf_template_data = load_adi_data(
    dataset_file,
    data_tag="object_stacked_05",
    psf_template_tag="psf_template",
    para_tag="header_object_stacked_05/PARANG")

psf_template = np.median(raw_psf_template_data, axis=0)

In [14]:
# we want the image to show the innermost 1.2 arcsec
print(1.2 / pixel_scale * 2)

88.30022075055187


In [15]:
# we cut the image to 91 x 91 pixel to be slightly larger than 1.2 arcsec
cut_off = int((science_data.shape[1] - 91) / 2)
science_data = science_data[:, cut_off:-cut_off, cut_off:-cut_off]

In [17]:
fwhm = get_fwhm(psf_template)
#fwhm = 3.6 # for most datasets
fwhm

3.6

In [18]:
# Use the exact SVD
svd_approx = -1

## 3. Save the parameters as json

In [19]:
# This is a residual from previous experiments for which we have optimized lambda
# but the effect is very small so we just run a few options (see the script)
best_lambda = 1000

In [21]:
parameter_dict = {
    "dataset_name" : dataset_name,
    "dit_psf" : dit_psf,
    "dit_science" : dit_science,
    "nd_scaling" : nd_scaling,
    "pixel_scale" : pixel_scale,
    "fwhm" : fwhm,
    "svd_approx" : svd_approx,
    "lambda_reg" : best_lambda,
    "num_epochs" : 500}

In [146]:
with open(root_dir / Path("30_data/" + dataset_name + ".json"), 'w', encoding='utf-8') as f:
    json.dump(parameter_dict, f, ensure_ascii=False, indent=4)